> **How to run this notebook (command-line)?**
1. Install the `ReinventCommunity` environment:
`conda env create -f environment.yml`
2. Activate the environment:
`conda activate ReinventCommunity`
3. Execute `jupyter`:
`jupyter notebook`
4. Copy the link to a browser


# `REINVENT 3.2`: `DRD2 use-case`
Due to the large number of possible settings and intrinsic mechanics, the learning curve for using `REINVENT` in a project setting can be quite steep. This notebook will illustrate

* how to prepare a project (considerations, data, goals),
* how to come up with a meaningful scoring function definition,
* how to monitor a job and
* how to judge the results of a run.

Technically, we will use the reinforcement learning running mode (see also respective notebook), but using a real dataset as starting point: the `DRD2` dataset.

## Starting a new project
In the beginning we usually start with gathering information: What do we know about the target? Do we have an idea where to start in chemical space? Do we plan to do *exploration* (find new scaffolds) or *exploitation* (find solutions around a known scaffold) in this project? What are the boundaries we want to set for the solutions in terms of e.g. physico-chemical properties, chemical moieties we would like to avoid or what is the target size of the molecules? Do we have enough data to e.g. generate a predictive (QSAR) model?

These considerations vary from one project to another, so it is impossible to formulate general rules. There is a large number of parameters that one can set to fine-tune a run with `REINVENT`, but most of them can be kept at their respective default values.

### Setting out the goals
For our `DRD2` use-case, let us assume that we have spoken to the lead chemist of the project and scribbled down a number of constraints for our molecules. They asked us to set up a `REINVENT` run that produces compounds which:

1. are likely to bind to `DRD2`,
2. are diverse (i.e. have different scaffolds),
3. do not have more than 6 hydrogen-bond donors and not more than 9 rotatable bonds and
4. avoid certain chemical moieties that are considered unfavourable.
5. Moreover, we have a couple of chemical compounds that we consider good lead for further developments, e.g. from a previous study or an experimental assay.

In general, our hypothetical chemists are open to new compound suggestions and do not necessarily want to find new solutions around specific scaffolds. This means, we are looking at an *exploration* scenario (rather than an *exploitation* one). Note, that it is possible to enforce molecules that satisfy one or more substructure matches (defined as `SMILES` or `SMARTS`), see the respective notebooks for details.

The data we are provided with is a dataset of more than 300000 molecules (in this case public data), which have been annotated to be either active ('1') or inactive ('0'). You can find the respective sets (already split into a training and a test set) in the `notebooks/data` subfolder.

Based on this information, our task now is to formulate a scoring function definition that will reward ("score") molecules that our chemists like with a better score than other molecules. This means we will break down the different requirements into `scoring function components`, assign a weight and recombine them into our scoring function. There are many more components available than the ones discussed here, so have a look at the other notebooks and the example `JSON` files in the `REINVENT` repository.

Before we start diving into this, though execute the following code blocks to set the configurations as required.

In [1]:
# load dependencies
import os
import re
import json
import tempfile

# --------- change these path variables as required
reinvent_dir = os.path.expanduser("~/Desktop/Reinvent")
reinvent_env = os.path.expanduser("~/miniconda3/envs/reinvent.v3.2")
output_dir = os.path.expanduser("~/Desktop/REINVENT_Use-Case-DRD2_demo")

# --------- do not change
# get the notebook's root path
try: ipynb_path
except NameError: ipynb_path = os.getcwd()

# if required, generate a folder to store the results
try:
    os.mkdir(output_dir)
except FileExistsError:
    pass

## Setting up the components
Once we have decided on the conditions along which we aim to design new molecules, we have to express them in terms of *scoring function components*. Each of the components will look independently at the compounds generated and report a score between `0` (very "bad") and `1` (very "good"). These individual scores are combined into a *total score* that will be used to calculate the loss and update the weights, thus "steering" the agent in chemical space. The components usually have parameters, so we decided to store them and their configurations in a `JSON` file for convenience. For our mock-project `DRD2` we will use the following components, but there are many more you might find useful. Have a look at the other notebooks and the example JSON files in the `REINVENT` repository.

### Predictive model (activity)
The first requirement is to generate molecules that show activity. As structural components (`ROCS`, docking, ...) are not part of this release, our only option is to build a predictive model that will tell `REINVENT` whether it considers a molecule as being promising or not. As of now, `REINVENT` only supports pickled [`scikit learn`](https://scikit-learn.org/stable/model_selection.html) models and based on the data we have available, we decide to build a classification model (you can find the "drd2.pkl" file in the `notebooks/data` subfolder). The models should accept one of the molecular fingerprint supported by `REINVENT`. The details of our model and how it was derived, are shown below:

*For the DRD2 model, an activity prediction model (APM) was used. This APM was trained on both the active and inactive compounds of the ExCAPE DRD2 modulator set. Stereochemistry was stripped from all compounds in the dataset and they were represented in their cannonical form by using RDKit. The resulting duplicates were removed. Data was split to test and training sets with a stratified split and  the compounds were represented with ECFP6 fingerprint (radius 3) hashed to 2048 bits. A Scikit-learn Random Forest Classifier (RF) model was trained to discriminate active from inactive compounds. Optuna was used for finding the optimal hyperparameters with a 5-fold cross validation. The performance of the resulting model in terms of area under the curve (AUC) was 0.945.*

Optuna citation:
Takuya Akiba, Shotaro Sano, Toshihiko Yanase, Takeru Ohta, and Masanori Koyama. 2019. Optuna: A Next-generation Hyperparameter Optimization Framework. https://arxiv.org/abs/1907.10902

Once we have built our model, we can formulate a scoring function component. As we decide to put a lot of emphasis on this component (and built our model on a lot of data, which boosts our confidence in the predictions), we will use a high weight for this component. Also, as this is a classification model and the probablility output is used, we do not need to scale the model's output to values between `0` and `1`. For other components, we will need to specify a score transformation, though. You can use a given `component_type` more than once (with different names), for example if we had another model predicting the toxicity of compounds in humans, we could simply add it as a new component.

In [2]:
component_DRD2_prediction = {
  "component_type": "predictive_property",
  "name": "DRD2_pred_activity",
  "weight": 7,
  "specific_parameters": {
    "model_path": os.path.join(ipynb_path, "models/drd2.pkl"),
    "scikit": "classification",
    "descriptor_type": "ecfp_counts",
    "size": 2048,
    "radius": 3,
    "use_counts": True,
    "use_features": True,
    "transformation": {
        "transformation_type": "no_transformation"
    }
  }
}

### Diversity
As mentioned above, in this project we are interested in a diverse set of molecular ideas. In order to force the agent to explore different areas in the chemical space, we can define a `diversity filter`. In effect, we will penalize families of molecules after a while: If the agent keeps sampling compounds with the same scaffold, the "scaffold bins" will fill up and once full every molecule with this scaffold will receive a score of `0`, thus "motivating" the agent to move on. We will use the standard parameters below, but feel free to adapt them (a detailed description is available in the other notebooks and the publication).

In [3]:
diversity_filter = {
  "name": "IdenticalMurckoScaffold",
  "nbmax": 25,
  "minscore": 0.4,
  "minsimilarity": 0.4
}

### Incept "good" ideas
Sometimes we want to provide the agent with a couple of (hopefully) good ideas to consider. If these molecules score well, the agent might pick up the trail and generate molecules around those initial ideas scoring ever more highly. There are three parameters:
1. `smiles`: A list of `SMILES` that will be subjected to the scoring function when the run starts.
2. `memory_size`: The scores for the `SMILES` are ordered from high to low and the `memory_size` best will be retained.
3. `sample_size`: At every epoch, a `sample_size` number of `SMILES` from the memory is randomly sampled and added to the batch of `SMILES` generated in that epoch.

For example, below we add 38 `SMILES` to the inception and for each molecule the score is calculated. The "Top-20" are stored in the memory and 5 of them are added to the first four epochs, respectively (until the memory is exhausted). Note, that one can set the `memory_size` to be greater or equal to the number of `SMILES` to use all the `SMILES` provided.

In [4]:
inception = {
  "memory_size": 20,
  "sample_size": 5,
  "smiles": [
    "OCCN1CCN(CCCN2c3ccccc3Sc3ccc(Cl)cc32)CC1",
    "C=CCN1CCC2c3cccc(OC)c3CCC21",
    "CCCN(Cc1ccc(OC)cc1)C1CCc2c(cccc2OC)C1",
    "N#Cc1ccc2[nH]c(CN3CCN(c4ccc(Cl)c(Cl)c4)CC3)cc2c1",
    "O=C(CCCN1CCN(c2ncccn2)CC1)c1ccc(F)cc1",
    "Cc1ccc2c(c1)C(N1CCN(C)CC1)=Nc1cccnc1N2",
    "COc1ccc(-c2cccc(CN3CCN(c4ncccn4)CC3)c2)cc1",
    "Oc1[nH]cc2ccc(OCCCCN3CCN(c4cccc5cccc(F)c45)CC3)c(Cl)c12",
    "CCCN1CCN(c2cccc(OC)c2)CC1",
    "O=C(Nc1cccc(SC(F)(F)F)c1)N1CCC(N2CCC(n3c(O)nc4c(F)cccc43)CC2)CC1",
    "CC1CCN(C2=Cc3cc(Cl)ccc3Cc3ccccc32)CC1",
    "CCCN1CCOC2Cc3c(O)cccc3CC21",
    "c1ccc(N2CCN(C3CCC(Nc4ncccn4)CC3)CC2)cc1",
    "O=S(=O)(NC1CCC(N2CCC(c3ccccc3OCC(F)(F)F)CC2)CC1)c1ccc(OC(F)F)cc1",
    "COc1ccccc1N1CCN(CCCCc2cc(-c3ccccc3)no2)CC1",
    "c1ccc(CCC2CCN(Cc3c[nH]c4ncccc34)CC2)cc1",
    "CC1CN(c2cccc3cc(F)ccc23)CCN1CCC1OCCc2c1sc(C(N)=O)c2Cl",
    "COc1ccccc1N1CCCN(CCCCNC(=O)c2cc3ccccc3o2)CC1",
    "O=C1CCc2ccc(OCCCCN3CCN(c4ccccc4C(F)(F)F)CC3)cc2N1",
    "CCCCCCCCCC(=O)N1c2ccc(Cl)cc2N=C(N2CCN(C)CC2)c2ccccc21",
    "O=C1N(CCN2CCC(C(F)(F)F)CC2)CCN1c1cccc(Cl)c1",
    "CN1CCc2cccc3c2C1Cc1cccc(-c2c(OS(=O)(=O)C(F)(F)F)cccc2OS(=O)(=O)C(F)(F)F)c1-3",
    "Cc1nn(C2CCN(Cc3cccc(C#N)c3)CC2)cc1-c1ccccc1",
    "COc1ccc(-c2cc3c(O)n(CCN4CCN(c5ccccc5Cl)CC4)c(O)nc-3n2)cc1",
    "OC1(c2cccc(Cl)c2Cl)CCN(Cc2c[nH]c3ccccc23)CC1",
    "FC(F)(F)c1ccc2c(c1)N(Cc1ccc(CNc3cccc(Oc4ccccc4)c3)cc1)c1ccccc1S2",
    "CCCSc1ccccc1N1CCN(CCCOc2ccc(-c3nc4ccccc4[nH]3)cc2)CC1",
    "O=S(=O)(NCCCCN1CCN(c2cccc(Cl)c2Cl)CC1)c1cc2ccccc2cn1",
    "COc1ccc(CN2CCN(CC(=O)N3c4ccccc4CC3C)CC2)cc1",
    "CC1Cc2ccccc2N1C(=O)CC1CCN(Cc2ccc(Cl)cc2)CC1",
    "CC(C)c1ccccc1N1CCN(CCCCCC(=O)NCc2ccccc2)CC1",
    "Cc1ncoc1-c1nnc(SCCCN2CCc3cc4nc(C5CC5)oc4cc3CC2)n1C",
    "O=C1CC(c2ccccc2)c2cccc(CCN3CCN(c4nsc5ccccc45)CC3)c2N1",
    "O=C1NCN(c2ccccc2)C12CCN(CCCOc1ccc(F)cc1)CC2",
    "CCON=C(CCN1CCN(c2nccs2)CC1)c1ccccc1",
    "Cc1ccc2c(-c3nnc(SCCCN4CCc5cc6nc(-c7cc(C)nn7C)oc6cc5CC4)n3C)cccc2n1",
    "CCCN(CCCCNC(=O)c1ccc(-c2ccccc2)cc1)C1Cc2cccn3ncc(c23)C1",
    "Cc1ccc(OCCNCCCOc2ccccc2)cc1"
  ]
}

### Molecular properties
There is a whole set of (physico-chemical) properties that can be set as scoring function components. Here, we will restrict ourselves to the numbers of hydrogen-bond donors and rotatable bonds. Note, that we need to define score transformations here to bring the resulting values into the 0-1 range explained above. Have a look at the `Score_Transformation` notebook for a detailed explanation. Normally, you will have to think about the range of values you would expect from a given component and then define the score transformation such, that values you consider "good" receive a transformed score close to `1` (and "bad" ones should be on the low side, closer to `0`). You are free to chose any (informative) name for your components, it is the `component_type` that specifies its nature. Typically, the `specific_parameters` block contains settings that only apply to this component (often a transformation, but also those that are required for this type only such as a descriptor definition for predictive models).

In [5]:
component_rotatable_bonds = {
  "component_type": "num_rotatable_bonds",
  "name": "Number of rotatable bonds",
  "weight": 1,
  "specific_parameters": {
    "transformation": {
        "transformation_type": "step",
        "high": 9,
        "low": 0
    }
  }
}

component_num_hbd = {
  "component_type": "num_hbd_lipinski",
  "name": "HB-donors (Lipinski)",
  "weight": 1,
  "specific_parameters": {
    "transformation": {
        "transformation_type": "step",
        "high": 6,
        "low": 0
    }
  }
}

### Avoid certain chemical groups
Usually, we want to avoid certain chemical moieties as they are associated with toxicity or unstable chemistry. Another reason could be that we already know that a certain chemical group will score very well and we want to explore new areas in the chemical space. We can achieve all this by specifying what we call `custom alerts`, i.e. a list of `SMARTS` which will drastically reduce the score of a compound exhibiting one or more matches. Note, that the impact of `custom alerts` is strong: Instead of simply adding a score component's contribution, a match with `custom alerts` will reduce a compound's total score by half (and will not affect it otherwise). For example, if a molecule receives a total score of 0.73, but happens to match a `custom alert` the score reported back to inform the agent will be 0.365 - this is necessary for the penalization to have enough impact.

In [6]:
component_custom_alerts = {
  "component_type": "custom_alerts",
  "name": "Custom_alerts",
  "weight": 1,
  "specific_parameters": {
      "smiles": [
        "[*;r8]",
        "[*;r9]",
        "[*;r10]",
        "[*;r11]",
        "[*;r12]",
        "[*;r13]",
        "[*;r14]",
        "[*;r15]",
        "[*;r16]",
        "[*;r17]",
        "[#8][#8]",
        "[#6;+]",
        "[#16][#16]",
        "[#7;!n][S;!$(S(=O)=O)]",
        "[#7;!n][#7;!n]",
        "C#C",
        "C(=[O,S])[O,S]",
        "[#7;!n][C;!$(C(=[O,N])[N,O])][#16;!s]",
        "[#7;!n][C;!$(C(=[O,N])[N,O])][#7;!n]",
        "[#7;!n][C;!$(C(=[O,N])[N,O])][#8;!o]",
        "[#8;!o][C;!$(C(=[O,N])[N,O])][#16;!s]",
        "[#8;!o][C;!$(C(=[O,N])[N,O])][#8;!o]",
        "[#16;!s][C;!$(C(=[O,N])[N,O])][#16;!s]"
         ]
      }
}


## Assembling the scoring function
Now that we have defined the individual components, it is time to assemble the scoring function. We have two choices here, `custom_sum` and `custom_product`. For the difference in the mathematical definition, we refer to the publication, but from a practical point of view it appears the former is a bit more "permissive", i.e. if a molecule receives a high component score for all but one of the components, it will still have a reasonably high total score. But `custom_product` will penalize such a molecule more harshly, making learning a bit more difficult. This could be what you want if you already have a highly focused agent, but as a safe default choice we recommend to use `custom_sum`.

In [7]:
scoring_function = {
  "name": "custom_sum",
  "parallel": True,
  "parameters": [
    component_DRD2_prediction,
    component_num_hbd,
    component_rotatable_bonds,
    component_custom_alerts
  ]
}

Note, that we add the `diversity_filter` later (outside of the scoring function in a block called `parameters`), as it is not really a score component (even though it has an impact on scoring the compounds).

## Complete the `JSON` configuration
There are still a couple of parameters we need to set in order to specify the run. These are set outside of the scoring function, so below you see the torso structure of a `REINVENT` configuration (with only the version number, the `run_type` and the `scoring_function` which we generated above) which we will enrich step-by-step. Remember that we will write out the dictionary we construct here as a `JSON` file in the end, so you can have a look at it.

In [8]:
configuration = {
  "version": 3,
  "run_type": "reinforcement_learning",
  "model_type": "default",
  "parameters": {
    "scoring_function": scoring_function
  }
}

### Add the diversity filter and the inception blocks
Here we add the diversity filter and inception blocks we specified above.

In [9]:
configuration["parameters"]["diversity_filter"] = diversity_filter
configuration["parameters"]["inception"] = inception

### Add run parameters
In addition, we need to specify a couple of settings (such as which `Prior` to use). We refer to the respective notebooks for details. The most important setting is the number of epochs `n_steps`: For illustrative purposes, we will set it to a mere 300 but usually you would set it to a range of 500-1500.

In [10]:
configuration["parameters"]["reinforcement_learning"] = {
    "prior": os.path.join(ipynb_path, "models/random.prior.new"),
    "agent": os.path.join(ipynb_path, "models/random.prior.new"),
    "n_steps": 300,
    "sigma": 128,
    "learning_rate": 0.0001,
    "batch_size": 128,
    "margin_threshold": 50
}

### Add the `logging` block
The final block we have to provide will specify where to store the output and so on. Usually, it will suffice to change `logging_path`, `result_folder` and `job_name`.

In [11]:
configuration["logging"] = {
    "sender": "http://127.0.0.1",
    "recipient": "local",
    "logging_frequency": 0,
    "logging_path": os.path.join(output_dir, "progress.log"),
    "result_folder": os.path.join(output_dir, "results"),
    "job_name": "Use-case DRD2 Demo",
    "job_id": "demo"
}

We now have successfully filled the dictionary and will write it out as a `JSON` file in the output directory. Please have a look at the file before proceeding in order to see how the paths have been inserted where required and the `dict` -> `JSON` translations (e.g. `True` to `true`) have taken place.

In [12]:
# write the configuration file to the disc
configuration_JSON_path = os.path.join(output_dir, "DRD2_config.json")
with open(configuration_JSON_path, 'w') as f:
    json.dump(configuration, f, indent=4, sort_keys=True)

## Run `REINVENT`
Now it is time to execute `REINVENT` locally. Note, that depending on the number of epochs (steps) and the execution time of the scoring function components, this might take a while. As we have only specified a low number of epochs and all components should be fairly quick, this should not take too long in our case though.

The command-line execution looks like this:
```
# activate envionment
conda activate reinvent.v3.2

# execute REINVENT
python <your_path>/input.py <config>.json
```

In [13]:
%%capture captured_err_stream --no-stderr

# execute REINVENT from the command-line
!{reinvent_env}/bin/python {reinvent_dir}/input.py {configuration_JSON_path}

In [14]:
# print the output to a file, just to have it for documentation
with open(os.path.join(output_dir, "run.err"), 'w') as file:
    file.write(captured_err_stream.stdout)

# prepare the output to be parsed
list_epochs = re.findall(r'INFO.*?local', captured_err_stream.stdout, re.DOTALL)
data = [epoch for idx, epoch in enumerate(list_epochs) if idx in [1, 150, 299]]
data = ["\n".join(element.splitlines()[:-1]) for element in data]

We have started with a *naive* prior as the initialization for the agent. Hopefully, we see that at the end of the reinforcement learning, the agent knows much better how to produce compounds that receive higher scores (based on our scoring function) than in the beginning. The following block will print the output for epochs 1, 150 and 299, respectively. We see that the "Score" (which is the average over all compounds) increases from 0.3156, over 0.4395, to 0.4628.

In [15]:
for element in data:
    print(element)

INFO     
 Step 0   Fraction valid SMILES: 98.4   Score: 0.3156   Time elapsed: 0   Time left: 0.0
  Agent     Prior     Target     Score     SMILES
-29.35    -29.35    -29.35      0.00      c1(S(=O)(C)=O)ccc(-c2c3[nH]c(C)c(CC(C)C)nc3ccc2)cc1
-29.12    -29.12    -29.12      0.00      C1Cc2cc(OC)c(O)cc2C1C(C(O)=O)C(C)C
-32.69    -32.69     28.64      0.48      [O-][N+](c1cc2c(cc1[N+](=O)[O-])n(C1OC(CO)C(O)C(O)C1O)c1c2cc(F)cc1)=O
-33.58    -33.58     29.48      0.49      CNCC(ON=C(c1ccc(Br)cc1)Cn1ncnc1)c1c(Cl)cccc1
-22.54    -22.54     49.98      0.57      C1C(NC(c2oc([N+](=O)[O-])cc2)=O)CCS1(=O)=O
-21.59    -21.59     47.42      0.54      c1c(C(Nc2c(C)nn(C)c2C)=O)ccc(Cl)c1
-21.95    -21.95     43.31      0.51      c1(C(N2CCN(c3c(=NC4CC4)[nH]c4ccccc4n3)CC2)=O)ccc(C(F)(F)F)cc1
-28.68    -28.68     34.63      0.49      C1(=O)c2ccccc2C(=O)C(C(=O)c2c(Cl)nc3ccc(Br)cc3c2)C1=O
-42.41    -42.41    -42.41      0.00      C1(C#N)C(c2ccc(OC(=O)C)cc2)C(C(=O)OCC)=C(CN2CCN(c3ccccc3)CC2)Oc2cc(OC)ccc21
-

## Analyse the results
In order to analyze the run in a more intuitive way, we can use `tensorboard`:

```
# go to the root folder of the output
cd <your_path>/REINVENT_RL_demo

# make sure, you have activated the proper environment
conda activate reinvent.v3.0

# start tensorboard
tensorboard --logdir progress.log
```

Then copy the link provided to a browser window, e.g. "http://workstation.url.com:6006/". The following figures are exmaple plots - remember, that there is always some randomness involved. In `tensorboard` you can monitor the individual scoring function components.

The results folder will hold four different files: the agent (pickled), the input JSON (just for reference purposes), the memory (highest scoring compounds in `CSV` format for the last epoch) and the scaffold memory (in `CSV` format, with the scaffolds and epoch numbers). 

In [16]:
!head -n 3 {output_dir}/results/scaffold_memory.csv

Step,Scaffold,SMILES,DRD2_pred_activity,HB-donors (Lipinski),Number of rotatable bonds,Custom_alerts,raw_DRD2_pred_activity,raw_HB-donors (Lipinski),raw_Number of rotatable bonds,total_score,ID
58.0,N=c1n(Cc2ccccc2)c2ccccc2n1S(=O)(=O)c1ccccc1,N=c1n(Cc2ccccc2)c2c(Cl)cccc2n1S(=O)(=O)c1ccc(F)c(F)c1,0.5164815187454224,1.0,1.0,1.0,0.5164815187454224,1.0,4.0,0.6239300966262817,Use-case DRD2 Demo_0
298.0,N=c1cc(-c2c[nH]nc2-c2ccccc2)cc[nH]1,CCn1cc(-c2cc(C)[nH]c(=NC(C)C)c2)c(-c2cc(Cl)cc(Cl)c2)n1,0.5127694010734558,1.0,1.0,1.0,0.5127694010734558,1.0,4.0,0.6210429072380066,Use-case DRD2 Demo_1


The next step is to gather all the ideas generated by `REINVENT` and decide which ones to move on with (post-processing). It could also be that another run has do be made with a different scoring function, but this is up to the user to decide. Ultimately, we have generated compounds that are predicted to have at least some potency for `DRD2`, which we can report back to our hypothetical chemist for further discussion.